In [ ]:
import numpy as np
import pandas as pd

In [ ]:
D4AI_list = pd.read_excel('Global_call_invite.xls')

In [ ]:
D4AI_list.head()

In [ ]:
D4AI_list['number_of_meetings']=0
D4AI_list['max_meetings']=100
D4AI_list.head()

In [ ]:
leads = D4AI_list[D4AI_list['Call Lead']=='x']

In [ ]:
leads['Studio'].value_counts()

In [ ]:
studios = set(D4AI_list['Studio'].values)
studios

In [ ]:
hours_diff = pd.read_csv('time_zones.csv')
hours_diff

In [ ]:
(hours_diff['hours_diff'] + 9)%24

In [ ]:
possible_groups = []

full_df = []
hour = []
for i in range(24):
    times = (hours_diff['hours_diff'] + i)%24
    
    #get the ones that are within working hours
    working_hours_bool = (times>8) & (times<18)
    working_studios = hours_diff[working_hours_bool]
    working_studios['time'] = times[working_hours_bool]
    hour.append(i)
#     print(working_studios)
#     print(' ')
    possible_groups.append(frozenset(working_studios['Studio']))
    full_df.append(working_studios)
print(possible_groups)

In [ ]:
i

In [ ]:
studio_pairings = set(possible_groups)

In [ ]:
studio_pairings

In [ ]:
actual_groups = []
for group in studio_pairings:
    if len(group)>2:
        actual_groups.append(group)

In [ ]:
actual_groups

In [ ]:
mapping_dict = {}
# for group in actual_groups:
#     for entry in group:

for studio in studios:
    if studio not in mapping_dict:
        mapping_dict[studio] = []
    for group in actual_groups:
        if studio in group:
            group = np.array([stud for stud in group])
            group = group[group!=studio]
            current_list = mapping_dict[studio]
            for entry in group:
                current_list.append(entry)
            mapping_dict[studio] = current_list

In [ ]:
poss = {}
for studio in mapping_dict:
    matches = mapping_dict[studio]
    possibilities = set(matches)
    poss[studio] = possibilities

In [ ]:
poss

In [ ]:
actual_groups

In [ ]:
from itertools import combinations

In [ ]:
comb = set(combinations(studios, 3))
comb 

In [ ]:
possible_groups = []

hour = []
comb_list = list(comb)
stored_dict = {}
for i in range(24):
    times = (hours_diff['hours_diff'] + i)%24
    
    #get the ones that are within working hours
    working_hours_bool = (times>8) & (times<18)
    working_studios = hours_diff[working_hours_bool]
    working_studios['time'] = times[working_hours_bool]
    hour.append(i)
    working_studios_set = set(working_studios.Studio.values)
    comb_bool = [sub for sub in comb if set(sub).issubset(working_studios_set)]
    if len(comb_bool)>0:
        for c in comb_bool:
            string = c[0]+'_'+ c[1]+'_'+c[2]
            if string in stored_dict:
                time_list = stored_dict[string]
                time_list.append(i)
                stored_dict[string] = time_list
            else:
                stored_dict[string] = [i]
                
stored_dict

In [ ]:
good_groups = []
for string in stored_dict:
    studios = string.split('_')
    times = stored_dict[string]
    #print(studios)
    if len(times)>0:
        good_groups.append(frozenset(studios))
good_groups = set(good_groups)

In [ ]:
actual_groups

In [ ]:
# check if any of those are in the actual groups
good_groups = []
for entry in comb:
    entry = frozenset(entry)
    for group in actual_groups:
        if entry.issubset(group):
            good_groups.append(entry)
good_groups = set(good_groups)

In [ ]:
good_groups

In [ ]:
def start_end_trio(trio:
    times = (hours_diff['hours_diff'] + i)%24

In [ ]:
leads = D4AI_list[D4AI_list['Call Lead']=='x']
weight = leads['max_meetings'] - leads['number_of_meetings']
selection = leads.sample(1, weights = weight)

selection_studio = selection.Studio.values[0]
selection_email = [selection.Email.values[0]]


In [ ]:
leads = D4AI_list[D4AI_list['Call Lead']=='x']
weight = leads['max_meetings'] - leads['number_of_meetings']
selection = leads.sample(1, weights = weight)

selection_studio = selection.Studio.values[0]
selection_email = [selection.Email.values[0]]

possibilities = []
for group in good_groups:
    if selection_studio in group:
        possibilities.append(group)

In [ ]:
import random
selected = []
while (sum(D4AI_list.number_of_meetings < 1) > 0):
    leads = D4AI_list[D4AI_list['Call Lead']=='x']
    weight = leads['max_meetings'] - leads['number_of_meetings']
    selection = leads.sample(1, weights = weight**5)

    selection_studio = selection.Studio.values[0]
    selection_email = [selection.index[0]]

   
    studio_group_index = random.randint(0, len(possibilities)-1)
    studio_group = possibilities[studio_group_index]
    other_studios = set(studio_group) - set([selection_studio])
    other_call_members = []
    for studio in other_studios:
        people = D4AI_list[((D4AI_list.Studio == studio) & (D4AI_list['Call Lead']!='x'))]
        weight = people['max_meetings'] - people['number_of_meetings']
        if sum(weight) == 0:
            people = D4AI_list[((D4AI_list.Studio == studio))]
            weight = people['max_meetings'] - people['number_of_meetings']
            # pick different studio grouping
            bad_studio_group_index = studio_group_index
            while studio_group_index == bad_studio_group_index:
                studio_group_index = random.randint(0, len(possibilities)-1)
                studio_group = possibilities[studio_group_index]
                other_studios = set(studio_group) - set([selection_studio])
                print('make new selection', other_studios)
                for studio in other_studios:
                    people = D4AI_list[((D4AI_list.Studio == studio) & (D4AI_list['Call Lead']!='x'))]
                    weight = people['max_meetings'] - people['number_of_meetings']
                
            
        selection = people.sample(1, weights = weight**7)
        selection_email.append(selection.index[0])
    print(selection_email)
    D4AI_list.loc[
                    selection_email, 'number_of_meetings'] += 1
    selected.append(selection_email)

In [ ]:
D4AI_list['number_of_meetings'].sum()

In [ ]:
# go through leads and assign groups for them, see how that turns out
D4AI_list['number_of_meetings']=0
D4AI_list['max_meetings']=5
D4AI_list.head()

In [ ]:
import random
selected = []
for index, selection in leads.iterrows():
    percentage_zero = D4AI_list[D4AI_list['number_of_meetings']==0].Studio.value_counts()/D4AI_list.Studio.value_counts()
    
    selection_studio = selection.Studio
    selection_email = [index]

    possibilities = []
    for group in good_groups:
        if selection_studio in group:
            possibilities.append(group)
            
    studio_group_index = random.randint(0, len(possibilities)-1) # weight this somehow
    studio_group = possibilities[studio_group_index]
    other_studios = set(studio_group) - set([selection_studio])
    other_call_members = []
    for studio in other_studios:
        people = D4AI_list[((D4AI_list.Studio == studio) & (D4AI_list['Call Lead']!='x'))]
        weight = people['max_meetings'] - people['number_of_meetings']
        if sum(weight) == 0:
            people = D4AI_list[((D4AI_list.Studio == studio))]
            weight = people['max_meetings'] - people['number_of_meetings']
            # pick different studio grouping
            bad_studio_group_index = studio_group_index
            while studio_group_index == bad_studio_group_index:
                studio_group_index = random.randint(0, len(possibilities)-1)
                studio_group = possibilities[studio_group_index]
                other_studios = set(studio_group) - set([selection_studio])
                print('make new selection', other_studios)
                for studio in other_studios:
                    people = D4AI_list[((D4AI_list.Studio == studio) & (D4AI_list['Call Lead']!='x'))]
                    weight = people['max_meetings'] - people['number_of_meetings']
                
            
        selection = people.sample(1, weights = weight**7)
        selection_email.append(selection.index[0])
    print(selection_email)
    D4AI_list.loc[
                    selection_email, 'number_of_meetings'] += 1
    selected.append(selection_email)

In [ ]:
def find_weight_for_studio_trio(trio, D4AI_list):
    
    total_weight = 0
    maxed = 1
    
    percentage_zero = D4AI_list[D4AI_list['number_of_meetings']==0].Studio.value_counts()/D4AI_list.Studio.value_counts()
    percentage_zero.fillna(0, inplace=True)
    
    percentage_max = D4AI_list[D4AI_list['number_of_meetings']==D4AI_list['max_meetings']].Studio.value_counts()/D4AI_list.Studio.value_counts()
    percentage_max.fillna(0, inplace=True)
    

    for studio in trio:
        p_zero = percentage_zero.loc[studio] #percentage people with zero meetings
        if percentage_max[studio]>=1:
            maxed = 0
        total_weight += p_zero
    return total_weight*maxed

In [ ]:
percentage_zero = D4AI_list[D4AI_list['number_of_meetings']==0].Studio.value_counts()/D4AI_list.Studio.value_counts()
percentage_zero.fillna(0, inplace=True)
percentage_zero

In [ ]:
#what if we go through the people, then select the lead?

In [ ]:
import random
D4AI_list['number_of_meetings']=0
D4AI_list['max_meetings']=3
D4AI_list.head()

non_leads = D4AI_list[(D4AI_list['Call Lead']!='x')]
leads = D4AI_list[(D4AI_list['Call Lead']=='x')]

selected = []
for index, selection in leads.iterrows():
    selection_studio = selection.Studio
    selection_email = [index]
    print('selection studio', selection_studio)
    print('selection email ', selection_email)
    
    non_leads = D4AI_list[(D4AI_list['Call Lead']!='x')]
    leads = D4AI_list[(D4AI_list['Call Lead']=='x')]

    possibilities = []
    weights = []
    for group in good_groups:
        if selection_studio in group:
            possibilities.append(group)
            weights.append(100*find_weight_for_studio_trio(group, non_leads))
            
    weights = np.array(weights)
    studio_group = pd.DataFrame({'col':possibilities}).sample(weights=weights**10).values[0][0]
    print(pd.DataFrame({'col':possibilities}).sample(weights=np.array(weights)))
    print(' ')
    other_studios = set(studio_group) - set([selection_studio])

    for studio in other_studios:
        #print('studio', studio)
        #people = D4AI_list[((D4AI_list.Studio == studio) & (D4AI_list['Call Lead']!='x'))]
        people = non_leads[non_leads.Studio==studio]
        weight = (people['max_meetings'] - people['number_of_meetings'])**10

        people['weights'] = weight
        # print(people[people.number_of_meetings>0])
        # print(' ')
        selection = people.sample(1, weights = weight)
        selection_email.append(selection.index[0])

    D4AI_list.loc[
                    selection_email, 'number_of_meetings'] += 1
    selected.append(selection_email)

In [ ]:
D4AI_list['number_of_meetings'].hist()

In [ ]:
D4AI_list['number_of_meetings'].value_counts()

In [ ]:
D4AI_list[D4AI_list['number_of_meetings']==0]

In [ ]:
def find_weight_for_studio_trio(trio, D4AI_list):
    
    total_weight = 0
    maxed = 1
    
    percentage_zero = D4AI_list[D4AI_list['number_of_meetings']==0].Studio.value_counts()/D4AI_list.Studio.value_counts()
    percentage_zero.fillna(0, inplace=True)
    
    percentage_max = D4AI_list[D4AI_list['number_of_meetings']==D4AI_list['max_meetings']].Studio.value_counts()/D4AI_list.Studio.value_counts()
    percentage_max.fillna(0, inplace=True)

    print('percentage max', percentage_max)
    print('percentage zero ', percentage_zero)
    
    for studio in trio:
        p_zero = percentage_zero.loc[studio] #percentage people with zero meetings
        if percentage_max[studio]>=1:
            maxed = 0
        total_weight += p_zero
    return total_weight*maxed

def find_possible_studios(selection_studio, good_groups):
    possibilities = []
    weights = []
    for group in good_groups:

        if selection_studio in group:
            possibilities.append(group)
            weights.append(100*find_weight_for_studio_trio(group, non_leads))
            
    weights = np.array(weights)
    return possibilities, weights


In [ ]:
import random
D4AI_list['number_of_meetings']=0
D4AI_list['max_meetings']=3
D4AI_list.head()

non_leads = D4AI_list[(D4AI_list['Call Lead']!='x')]
leads = D4AI_list[(D4AI_list['Call Lead']=='x')]

selected = []
while (sum(D4AI_list.number_of_meetings < 1) > 0):
    non_leads = D4AI_list[(D4AI_list['Call Lead']!='x')]
    leads = D4AI_list[(D4AI_list['Call Lead']=='x')]
    weights = [0,0]
    
    while (sum(weights)==0):
        weight = (leads['max_meetings'] - leads['number_of_meetings'])**15
        selection = leads.sample(1, weights = weight)

        selection_studio = selection.Studio.values[0]
        selection_email = [selection.index[0]]

        possibilities, weights = find_possible_studios(selection_studio, good_groups)

    studio_group = pd.DataFrame({'col':possibilities}).sample(weights=weights**10).values[0][0]
    other_studios = set(studio_group) - set([selection_studio])

    for studio in other_studios:
        people = non_leads[non_leads.Studio==studio]
        weight = (people['max_meetings'] - people['number_of_meetings'])**15

        people['weights'] = weight
        selection = people.sample(1, weights = weight)
        selection_email.append(selection.index[0])
    D4AI_list.loc[
                    selection_email, 'number_of_meetings'] += 1
    selected.append(selection_email)
print(len(selected))

In [ ]:
D4AI_list['number_of_meetings'].hist()
plt.show()

In [ ]:
emails_all = []
studios_all = []
for row in selected:
    emails = []
    studios = []
    for index in row:
        person = D4AI_list.iloc[index]
        emails.append(person.Email)
        studios.append(person.Studio)
    emails_all.append(emails)
    studios_all.append(studios)

In [ ]:
studios_all

In [ ]:
emails_all